In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

ModuleNotFoundError: No module named 'selenium'

In [2]:
! pip install selenium


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 6.7/6.7 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 1.0 MB/s eta 0:00:00


In [2]:
# the website to be scrapped
website="https://www.instahyre.com/search-jobs/"

# get request to the url
opts=Options()
driver=webdriver.Chrome(opts)
driver.get(website)

# creating empty list for the data
data=[]

In [3]:
# taking total no of pages to be repeated for data collection
pages= 3

# running for loop for different pages upto n pages
for i in range(pages):
    print("Page", i+1)

    # making soup
    html_code=driver.page_source
    soup=BeautifulSoup(html_code,'html.parser')
    
    # all rows in one page
    rows=soup.find_all(class_='row cursor-pointer')
    
    # run loop for each row to get the data by clicking on the view tab
    for row in rows:
        link=row.find('a')['href']

        #printing each link of view tab
        print("directing to link :", link) 
        
        # taking content_driver as a driver for content for each row 
        content_driver = webdriver.Chrome(opts)
        content_driver.get(link)
        
        # making soup for content
        content_html_code = content_driver.page_source
        content_soup = BeautifulSoup(content_html_code, 'html.parser')
        
        # extracting required data from each content page
        
        # link itself contain the job_id https://www.instahyre.com/job-261410-admin-at-indihood-bangalore/
        job_id = int(link.split('-')[1])
        
        # text of tag is itself the job_title
        job_title = content_soup.find("h1").text.strip()
        
        #text of tag is itself the company_name
        company_name = content_soup.find(class_="company-name").text.strip()
        
        #text of tag is itself the experience
        experience = content_soup.find(class_="experience").text.strip()
        
        # text of tag is itself the job location
        job_locations = content_soup.find(class_="job-locations").text.strip().replace(experience, "")
        
        # text of tag is itself the job hr_name
        HR_name = content_soup.find(class_="rec-name").text.strip()
        
        # text of tag in respective attibute
        founded_in = content_soup.find("div", {"ng-if": "employer.company_founded"}).text.strip()
        
        # text of tag in respective attibute
        employees_count = content_soup.find("div", {"ng-if": "employer.employee_count"}).text.strip()
        
        # there are multiple skills in their tag
        skill = content_soup.find(class_="tags job-description-skills")
        
        if skill and skill.children:
            skill=skill.children
        else:
            skill=[skill]
        
        skills_list = []
        
        if skill is not None:
            for j in skill:
                j = j.text.strip()
                if (j and j != "\\n"):
                    skills_list.append(j)
                    
        if skills_list is None:
            skill="N/A"
        else:
            skill= ", ".join(skills_list)
            
        # appending extracted data to data list   
        data.append({"company_name" : company_name, "job_title" : job_title, "job_locations" : job_locations, 
                         "experience" : experience,"hr_name" : HR_name,"founded_in" : founded_in,
                     "employees_count" : employees_count, "skills" : skill, "job_id" : job_id })
        
        # Return to the previous page
        content_driver.back()
        content_driver.quit()
        
    # again clicking the view tab for another row
    next_page = driver.find_element(By.CSS_SELECTOR, '[ng-click="nextPage()"]').click()
    
# printing no of data and data    
print(len(data))
print("data",data)

# converting list into datframe
df = pd.DataFrame(data)
print(df)
           
df.to_excel('F://scrap_data//data_5.xlsx', columns=['company_name', 'job_title', 'job_locations', 'experience', 'hr_name', 
                                                    'founded_in', 'employees_count', 'skills','job_id'], index=False)   

Page 1
directing to link : https://www.instahyre.com/job-261410-admin-at-indihood-bangalore/
directing to link : https://www.instahyre.com/job-261457-customer-success-analyst-at-insightsoftware-hyderabad/


KeyboardInterrupt: 